In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [19]:
data = {'animal': ['dog', 'cat', 'bird'], 'age': [1, 2, 1],'owner': ['jacky', 'tom', 'terry']}
data = pd.DataFrame(data)

In [20]:
data.head()

,animal,age,owner
0,dog,1,jacky
1,cat,2,tom
2,bird,1,terry


## one-hot 列形式

In [21]:
# data = pd.get_dummies(data, prefix=['animal', 'owner'])  
#prefix是转换后加追加在列前边的字符串，需和需转换的列长度一致
data1 = pd.get_dummies(data)

In [22]:
data1.head()

,age,animal_bird,animal_cat,animal_dog,owner_jacky,owner_terry,owner_tom
0,1,0,0,1,1,0,0
1,2,0,1,0,0,0,1
2,1,1,0,0,0,1,0


## 标签值标准化

In [27]:
encoder = LabelEncoder()  
encoder.fit(data['animal'])  

LabelEncoder()

In [28]:
#标签个数
encoder.classes_

array(['bird', 'cat', 'dog'], dtype=object)

In [29]:
#标签值标准化
encoder.transform(data['animal'])

array([2, 1, 0])

In [30]:
#标准化标签值反转
encoder.inverse_transform([2, 1, 0])

array(['dog', 'cat', 'bird'], dtype=object)

## 处理法官信息

In [2]:
judge=pd.read_excel(os.path.join('../rawdata','两院法官人员信息.xlsx')) 

In [38]:
judge.head(3)

,id,姓名,法院,部门,法律职务,性别,行政职务,等级,是否员额法官,年龄,工作年限
0,1,杨青,0,5,9,0,17,8,0,24,NaN
1,2,凌文英,0,7,6,0,17,7,1,53,35.0
2,3,张雪琪,0,5,9,0,17,8,0,25,NaN


## 去掉法官重名的情况，将人名与Id一一对应

In [5]:
judge0=judge[judge['法院']=='重庆市江北区人民法院']
judge1=judge[judge['法院']=='重庆市渝北区人民法院']

In [6]:
#选出重复的名字：重庆市江北区人民法院：删除id==370
#渝北区人民法院：删除id==501,639,648,664,673,737,765,797,836
judge0[judge0['姓名'].duplicated(False)]

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官
369,370,陈瑶,重庆市江北区人民法院,司法警察大队,NaT,NaN,男,NaN,1995-01-13,NaN,2.0
384,385,陈瑶,重庆市江北区人民法院,金融审判庭,NaT,NaN,女,NaN,1987-11-16,NaN,NaN


In [7]:
judge0.drop([369],inplace=True)
judge1.drop([500,638,647,663,672,736,764,796,835],inplace=True)

/home/huipu/.local/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
def judgeToId(judge1):
    judge2id={}
    # id2judge={}
    for index, row in judge1.iterrows():
        judge2id[row[1]]=row[0]
    #     id2judge[row[0]]=row[1]
    return judge2id

In [9]:
judge2id0=judgeToId(judge0)
judge2id1=judgeToId(judge1)

### 将重复的部门归为一类

In [10]:
judge['部门'].replace('政治部（机关党委）','政治部（机关党委)',inplace=True)
judge['部门'].replace('监察','监察室',inplace=True)

In [12]:
encoder = LabelEncoder()  

## 法院标签值标准化

In [13]:
encoder.fit(judge['法院'])  

LabelEncoder()

In [14]:
encoder.classes_

array(['重庆市江北区人民法院', '重庆市渝北区人民法院'], dtype=object)

In [15]:
judge['法院']=encoder.transform(judge['法院'])

## 部门标签值标准化

In [16]:
encoder.fit(judge['部门'])  

LabelEncoder()

In [17]:
encoder.classes_

array(['两江鱼复人民法庭', '刑事审判庭', '司法警察大队', '后勤服务中心', '审判管理办公室（研究室）', '执行局',
       '政治部（机关党委)', '民事审判一庭', '民事审判三庭', '民事审判二庭', '民事财产保全中心', '洛碛人民法庭',
       '环境资源审判庭', '监察室', '立案庭（诉讼服务中心）', '统景人民法庭', '综合办公室', '茨竹人民法庭',
       '行政审判庭（综合审判庭）', '金融审判庭', '院领导'], dtype=object)

In [18]:
judge['部门']=encoder.transform(judge['部门'])

## 入院日期与出生日期处理

In [19]:
judge['年龄']=datetime.today().year-judge['出生日期'].dt.year
judge['工作年限']=datetime.today().year-judge['入院日期'].dt.year
judge.drop(['入院日期','出生日期'], axis=1,inplace=True)

## 法律职务

In [20]:
judge['法律职务'].fillna('无',inplace=True);

In [21]:
encoder.fit(judge['法律职务'])  

LabelEncoder()

In [22]:
encoder.classes_

array(['书记员', '其他审判辅助人员', '副庭长', '副院长', '助理审判员', '司法行政人员', '审判员',
       '审判委员会委员', '庭长', '无', '法官助理', '法警', '院长'], dtype=object)

In [23]:
judge['法律职务']=encoder.transform(judge['法律职务'])

## 性别

In [24]:
encoder.fit(judge['性别']) 

LabelEncoder()

In [25]:
encoder.classes_

array(['女', '男'], dtype=object)

In [26]:
judge['性别']=encoder.transform(judge['性别'])

## 行政职务

In [27]:
judge['行政职务'].fillna('无',inplace=True);
encoder.fit(judge['行政职务']) 

LabelEncoder()

In [28]:
encoder.classes_

array(['专职审委会委员', '主任', '主任科员', '其他', '副主任', '副主任科员', '副大队长', '副庭长',
       '副科长', '副调研员', '副院长', '大队长', '庭长', '执行局副局长', '执行局局长', '政治部（处）主任',
       '教导员', '无', '科员', '纪检组组长', '调研员', '院长'], dtype=object)

In [29]:
judge['行政职务']=encoder.transform(judge['行政职务'])

## 等级

In [30]:
judge['等级'].fillna('无',inplace=True);
encoder.fit(judge['等级']) 

LabelEncoder()

In [31]:
encoder.classes_

array(['一级法官', '三级法官', '三级高级法官', '二级法官', '二级高级法官', '五级法官', '四级法官',
       '四级高级法官', '无', '法官等级未定'], dtype=object)

In [32]:
judge['等级']=encoder.transform(judge['等级'])

## 是否员额法官

In [33]:
judge['是否员额法官'].replace(2.0,0,inplace=True)

In [34]:
judge['是否员额法官'].fillna(0,inplace=True);

In [35]:
judge['是否员额法官']=judge['是否员额法官'].astype('int64');